# On and Off Statistics

In [1]:
# Let's first load all required packages...
import os
import pandas as pd
import numpy  as np
import dtale

from config import *
from bball_stats import *
import tools

In [2]:
# game_id = 2004609  # 2 overtimes
# game_id = 1976446  # different names capitalisation - 6 players
# game_id = 2031329 # players come 
# game_id = 2004608  # Period 2 05.600 - B. Kuol (team 2) come out, but then in and out

# game_id = 1976449   # Makers goes in (for Martin) even though he is already there! So 4 players are left with Martin off
# game_id = 1976457   # same issue with Makers, he goes in even though he is on court!

game_id = 2031329

Process game and extract all data structures (e.g., play-by-plays, stints):

In [3]:
set_logging("INFO")

game_json = tools.get_json_data(game_id)
starters = get_starters(game_json, 1)
pbp_df = get_pbp_df(game_json)
stints = pbp_stints_extract(pbp_df, starters, 1)

result = build_game_stints_stats_df(game_id)
stats_df = result['stint_stats_df']
pbp_df = result['pbp_df']
team1 = result['teams'][0]
team2 = result['teams'][1]
stints1_df = result['stints_df'].query('tno == 1')
stints2_df = result['stints_df'].query('tno == 2')

stints1_df[['id', 'lineup']].to_records(index=False)

2022-07-17 20:58:46 WARNING Sub team 1 at 00:07:43 in period 4: outcoming players not in court: {'H. Besson'}
2022-07-17 20:58:46 WARNING Sub team 1 at 00:01:44 in period 4: incoming players already in court: {'O. Dieng'}
2022-07-17 20:58:46 WARNING Sub team 1 at 00:07:43 in period 4: outcoming players not in court: {'H. Besson'}
2022-07-17 20:58:46 WARNING Sub team 1 at 00:01:44 in period 4: incoming players already in court: {'O. Dieng'}
2022-07-17 20:58:46 WARNING Sub team 2 at 00:10:00 in period 4: incoming players already in court: {'C. Goulding'}
2022-07-17 20:58:46 WARNING Sub team 2 at 00:05:42 in period 4: incoming players already in court: {'M. Peatling'}
2022-07-17 20:58:46 WARNING Sub team 2 at 00:05:42 in period 4: outcoming players not in court: {'J. White', 'S. Ili'}


rec.array([( 1, list(['F. Delany', 'H. Besson', 'O. Dieng', 'W. McDowell-White', 'Y. Wetzell'])),
           ( 2, list(['F. Delany', 'J. Martin', 'O. Dieng', 'P. Siva', 'Y. Wetzell'])),
           ( 3, list(['F. Delany', 'H. Besson', 'J. Martin', 'P. Siva', 'S. Timmins'])),
           ( 4, list(['H. Besson', 'J. Martin', 'K. Galloway', 'P. Siva', 'S. Timmins'])),
           ( 5, list(['H. Besson', 'J. Martin', 'K. Galloway', 'P. Siva', 'Y. Wetzell'])),
           ( 6, list(['H. Besson', 'J. Martin', 'K. Galloway', 'W. McDowell-White', 'Y. Wetzell'])),
           ( 7, list(['F. Delany', 'H. Besson', 'J. Martin', 'W. McDowell-White', 'Y. Wetzell'])),
           ( 8, list(['F. Delany', 'H. Besson', 'P. Siva', 'W. McDowell-White', 'Y. Wetzell'])),
           ( 9, list(['H. Besson', 'J. Martin', 'P. Siva', 'W. McDowell-White', 'Y. Wetzell'])),
           (10, list(['F. Delany', 'J. Martin', 'O. Dieng', 'P. Siva', 'S. Timmins'])),
           (11, list(['F. Delany', 'J. Martin', 'O. Dieng', '

In [4]:
dtale.show(stints1_df)

Check if there are plays with no player assigned.

In principle, there should be none, but there are usually some, e.g., in rebounds; guess it's bc they missed the relevant player in the play?

In [5]:
# find plays with no player assigned! Should be empty 
pbp_df.query('player == "nan. nan" and ((tno == 1) or (tno == 2))')

,team_name,team_short_name,clock,s1,s2,lead,tno,period,periodType,pno,player,success,actionType,actionNumber,previousAction,qualifier,subType,scoring,stint1,stint2
483,New Zealand Breakers,Breakers,00:07:29,5,5,0,1,1,REGULAR,0,nan. nan,1,rebound,36,34,[team],offensive,0,1,1
436,Melbourne United,United,00:04:23,8,11,-3,2,1,REGULAR,0,nan. nan,1,rebound,96,94,[team],defensive,0,2,4
423,New Zealand Breakers,Breakers,00:03:29,11,15,-4,1,1,REGULAR,0,nan. nan,1,timeout,115,,[],full,0,3,5
330,New Zealand Breakers,Breakers,00:05:44,33,30,3,1,2,REGULAR,0,nan. nan,1,timeout,682,,[],full,0,7,7
310,Melbourne United,United,00:04:22,34,32,2,2,2,REGULAR,0,nan. nan,1,rebound,266,264,[team],defensive,0,8,8
277,New Zealand Breakers,Breakers,00:01:41,39,34,5,1,2,REGULAR,0,nan. nan,1,turnover,681,,[team],24sec,0,9,9
225,Melbourne United,United,00:07:31,45,45,0,2,3,REGULAR,0,nan. nan,1,rebound,383,380,[team],offensive,0,2,11
189,Melbourne United,United,00:03:43,56,54,2,2,3,REGULAR,0,nan. nan,1,timeout,425,,[],full,0,10,7
151,Melbourne United,United,00:00:55.400000,58,63,-5,2,3,REGULAR,0,nan. nan,1,rebound,463,462,[team],defensive,0,11,14
108,New Zealand Breakers,Breakers,00:08:04,64,71,-7,1,4,REGULAR,0,nan. nan,1,timeout,527,,[],full,0,13,15


In [6]:
# remove NaN players
players_1 = list(pbp_df.query('tno == 1')['player'].unique())
players_2 = list(pbp_df.query('tno == 2')['player'].unique())

players_1.remove('nan. nan')
players_2.remove('nan. nan')

players = players_1
players
# pbp_df.query('tno == 2').loc[players_2.isna()]

['Y. Wetzell',
 'F. Delany',
 'H. Besson',
 'O. Dieng',
 'W. McDowell-White',
 'P. Siva',
 'J. Martin',
 'S. Timmins',
 'K. Galloway',
 'R. Loe']

Next, we extract the stint numbers of each player:

In [7]:
def get_stints_player(stints_df: pd.DataFrame, player: str):
    mask = stints_df.apply(lambda x: player in x['lineup'], axis=1)

    return stints_df.loc[mask, 'id'].to_list()

# stints1_df
players_stints = dict({})
for x in players:
    players_stints[x] = get_stints_player(stints1_df, x)

players_stints

{'Y. Wetzell': [1, 2, 5, 6, 7, 8, 9, 13, 14, 15, 16],
 'F. Delany': [1, 2, 3, 7, 8, 10, 11, 14, 15],
 'H. Besson': [1, 3, 4, 5, 6, 7, 8, 9, 15, 16],
 'O. Dieng': [1, 2, 10, 11, 12, 13, 14, 15, 16],
 'W. McDowell-White': [1, 6, 7, 8, 9, 11, 12, 13, 14],
 'P. Siva': [2, 3, 4, 5, 8, 9, 10, 12, 13, 14, 15, 16],
 'J. Martin': [2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16],
 'S. Timmins': [3, 4, 10],
 'K. Galloway': [4, 5, 6],
 'R. Loe': [11, 12]}

In [8]:
# An example: player H. Besson with/without W. McDowell-White
x = 'H. Besson'
y = 'W. McDowell-White'

x_stints = set(players_stints[x])
y_stints = set(players_stints[y])

print(f"Stints where {x} played:", x_stints)
print(f"Stints where {y} played:", y_stints)
print(y)

# x_shoots_df = pbp_df.query('stint1 in @x_stints and player == @x and actionType in @SHOOTS_TYPES and success == 1')
x_shoots_df = pbp_df.query('player == @x and actionType in @SHOOTS_TYPES and success == 1')
# df.query('actionType == "2pt"')

x_shoots_df


Stints where H. Besson played: {1, 3, 4, 5, 6, 7, 8, 9, 15, 16}
Stints where W. McDowell-White played: {1, 6, 7, 8, 9, 11, 12, 13, 14}
W. McDowell-White


,team_name,team_short_name,clock,s1,s2,lead,tno,period,periodType,pno,player,success,actionType,actionNumber,previousAction,qualifier,subType,scoring,stint1,stint2
499,New Zealand Breakers,Breakers,00:08:43,3,2,1,1,1,REGULAR,14,H. Besson,1,3pt,14,,[],jumpshot,1,1,1
418,New Zealand Breakers,Breakers,00:02:27,13,15,-2,1,1,REGULAR,14,H. Besson,1,2pt,127,,[pointsinthepaint],layup,1,3,5
368,New Zealand Breakers,Breakers,00:09:17,23,21,2,1,2,REGULAR,14,H. Besson,1,freethrow,201,,[fastbreak],1of2,1,6,7
367,New Zealand Breakers,Breakers,00:09:17,24,21,3,1,2,REGULAR,14,H. Besson,1,freethrow,202,,[fastbreak],2of2,1,6,7
315,New Zealand Breakers,Breakers,00:04:42,34,32,2,1,2,REGULAR,14,H. Besson,1,freethrow,259,,"[fastbreak, fromturnover]",1of2,1,8,7
159,New Zealand Breakers,Breakers,00:01:50,58,61,-3,1,3,REGULAR,14,H. Besson,1,2pt,454,,[pointsinthepaint],jumpshot,1,11,14
87,New Zealand Breakers,Breakers,00:06:12,67,77,-10,1,4,REGULAR,14,H. Besson,1,3pt,562,,[],jumpshot,1,15,15
66,New Zealand Breakers,Breakers,00:04:25,71,84,-13,1,4,REGULAR,14,H. Besson,1,2pt,581,,[pointsinthepaint],jumpshot,1,15,7
49,New Zealand Breakers,Breakers,00:03:20,73,84,-11,1,4,REGULAR,14,H. Besson,1,2pt,604,,[fromturnover],floatingjumpshot,1,15,17


In [11]:
players = players_1

onoff_df = pd.DataFrame()  # Correlation matrix
for x in players:
    x_stints = set(players_stints[x])
    x_pbp_df = pbp_df.query('player == @x and actionType in @SHOOTS_TYPES and success == 1')
    c_3pt = x_pbp_df.query('actionType == "3pt"').shape[0]
    c_2pt = x_pbp_df.query('actionType == "2pt"').shape[0]
    c_1pt = x_pbp_df.query('actionType == "freethrow"').shape[0]
    x_pts = 2*c_2pt + 3*c_3pt + c_1pt
    for y in players:
        if x == y:
            v = -1
        else:
            y_stints = set(players_stints[y])
            together = x_stints.intersection(y_stints)
            alone = x_stints.difference(y_stints)
            # print(x, y, together, alone)

            together_df = x_pbp_df.query('stint1 in @together')
            c_2pt = together_df.query('actionType == "2pt"').shape[0]
            c_3pt = together_df.query('actionType == "3pt"').shape[0]
            c_1pt = together_df.query('actionType == "freethrow"').shape[0]
            together_pts = 2*c_2pt + 3*c_3pt + c_1pt

            alone_df = x_pbp_df.query('stint1 in @alone')
            c_2pt = alone_df.query('actionType == "2pt"').shape[0]
            c_3pt = alone_df.query('actionType == "3pt"').shape[0]
            c_1pt = alone_df.query('actionType == "freethrow"').shape[0]
            alone_pts = 2*c_2pt + 3*c_3pt + c_1pt

            v = together_pts - alone_pts

            print(f"Player {x} ({x_pts} pts) with player {y}: {together_pts} and without: {alone_pts}")
        onoff_df.loc[x, y] = v

onoff_df


,Y. Wetzell,F. Delany,H. Besson,O. Dieng,W. McDowell-White,P. Siva,J. Martin,S. Timmins,K. Galloway,R. Loe
Y. Wetzell,-1.0,7.0,7.0,-3.0,7.0,-1.0,-3.0,-7.0,-7.0,-7.0
F. Delany,8.0,-1.0,2.0,4.0,-4.0,8.0,4.0,-8.0,-8.0,-8.0
H. Besson,11.0,11.0,-1.0,5.0,-3.0,5.0,7.0,-11.0,-11.0,-15.0
O. Dieng,-3.0,3.0,-3.0,-1.0,-3.0,3.0,3.0,3.0,-3.0,-3.0
W. McDowell-White,2.0,2.0,2.0,-2.0,-1.0,-2.0,2.0,-2.0,-2.0,-2.0
P. Siva,1.0,7.0,7.0,1.0,-19.0,-1.0,19.0,-1.0,-7.0,-19.0
J. Martin,8.0,6.0,-4.0,4.0,-2.0,10.0,-1.0,-8.0,-14.0,-20.0
S. Timmins,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0
K. Galloway,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0
R. Loe,-2.0,-2.0,-2.0,2.0,2.0,2.0,2.0,-2.0,-2.0,-1.0


In [108]:
# stints1_df.iloc[3]['lineup']

[2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16]